In [1]:
import pandas as pd
import os
from tqdm import tqdm
import yaml
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import csv
import time
import numpy as np

with open("/home/ubuntu/work/therapeutic_accelerator/config/main.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
with open("../config/keys.yaml", "r") as f:
    keys = yaml.load(f, Loader=yaml.FullLoader)
    
bucket_path = os.path.join(config['paths']['root'], config['paths']['mount'])

In [2]:
engine = create_engine(f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres')

# Query to pull in corpus IDs

sql = '''
    SELECT * FROM abstracts;
'''

# Context manager to open the connection to the database and execute the query. 
with engine.connect() as conn:
    query = conn.execute(text(sql))
    
# Get all records and store in dataframe
abstracts = pd.DataFrame(query.fetchall())

abstracts.head()


,index,paperId,corpusId,abstract
0,None,39063bcddbd9decd4c8f5c5d3e8392280ab046ad,52273203,None
1,None,6ec7c156b4173ad7ca0dbc654da9267474644a41,23708908,Constitutive JAK/STAT3 signaling contributes t...
2,None,c856627242a754d2d756b32843523e6d7a089148,13232625,Summary: The current work characterizes young ...
3,None,d4c9b2fa2b760b5cf90ce8635a7dede5b4cd58a2,73484844,Ionotropic glutamate receptors (iGluRs) mediat...
4,None,7f13abe2c82bf0c66ca423e905d8f5967c4517b1,229159752,Aim The current pandemic of coronavirus diseas...


In [4]:
# Query to pull in corpus IDs
sql = '''
    SELECT * FROM attributes;
'''

# Context manager to open the connection to the database and execute the query. 
with engine.connect() as conn:
    query = conn.execute(text(sql))
    
# Get all records and store in dataframe
attributes = pd.DataFrame(query.fetchall())
attributes.head()

,externalids,url,title,authors,venue,publicationvenueid,year,referencecount,citationcount,influentialcitationcount,isopenaccess,s2fieldsofstudy,publicationtypes,publicationdate,journal,updated,id
0,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/39063bcd...,Large conformational changes of a highly dynam...,"{""{\""authorId\"": \""51285510\"", \""name\"": \""Isa...",Communications Biology,069e05e7-ca35-41d0-a8c7-bdc9ec6a82af,2018.0,28,13,0,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2018-09-03,"{""name"": ""Communications Biology"", ""pages"": nu...",2023-03-31T01:28:37.897Z,5692596
1,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/6ec7c156...,Drug sensitivity profiling identifies potentia...,"{""{\""authorId\"": \""5699001\"", \""name\"": \""H. K...",OncoTarget,106ecc86-1801-4b36-a05c-d36db69a968c,2017.0,46,24,0,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2017-10-31,"{""name"": ""Oncotarget"", ""pages"": ""97516 - 97527...",2023-04-05T06:18:15.837Z,5692609
2,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/c8566272...,Men's memory for women's sexual-interest and r...,"{""{\""authorId\"": \""4425040\"", \""name\"": \""T. T...",None,None,2011.0,36,13,0,True,"{""{\""category\"": \""Psychology\"", \""source\"": \...",{Review},2011-09-01,"{""name"": ""Applied Cognitive Psychology"", ""page...",2023-04-05T08:18:02.240Z,5692611
3,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/d4c9b2fa...,The Reactive Plasticity of Hippocampal Ionotro...,"{""{\""authorId\"": \""2225106\"", \""name\"": \""A. M...",International Journal of Molecular Sciences,8506a01a-40b8-4e6f-bbb8-ce2492139c15,2019.0,97,15,1,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...","{Review,JournalArticle}",2019-02-27,"{""name"": ""International Journal of Molecular S...",2023-05-20T17:20:54.230Z,5692615
4,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/7f13abe2...,High Mortality Rate in Adult COVID-19 Inpatien...,"{""{\""authorId\"": \""7685173\"", \""name\"": \""Saee...",Journal of Multidisciplinary Healthcare,12e5e550-28bd-401c-a348-f347956deec4,2020.0,37,17,1,True,"{""{\""category\"": \""Medicine\"", \""source\"": \""s...","{JournalArticle,Review}",2020-12-01,"{""name"": ""Journal of Multidisciplinary Healthc...",2023-05-23T13:34:07.581Z,5692620


In [25]:
"52273203" in attributes['id'].tolist()

False

In [26]:
attributes['id']

0         5692596
1         5692609
2         5692611
3         5692615
4         5692620
           ...   
729812    5439867
729813    5439869
729814    5439890
729815    5439907
729816    5439910
Name: id, Length: 729817, dtype: string

In [15]:
attributes.dtypes

externalids                  string
url                          string
title                        string
authors                      string
venue                        string
publicationvenueid           string
year                          Int64
referencecount                Int64
citationcount                 Int64
influentialcitationcount      Int64
isopenaccess                boolean
s2fieldsofstudy              string
publicationtypes             string
publicationdate              string
journal                      string
updated                      string
id                           string
dtype: object

In [23]:
abstracts.corpusId == attributes.id

0

In [16]:
abstracts.merge(attributes, how = 'left', left_on = 'corpusId', right_on = 'id')

,index,paperId,corpusId,abstract,externalids,url,title,authors,venue,publicationvenueid,...,referencecount,citationcount,influentialcitationcount,isopenaccess,s2fieldsofstudy,publicationtypes,publicationdate,journal,updated,id
0,None,39063bcddbd9decd4c8f5c5d3e8392280ab046ad,52273203,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,None,6ec7c156b4173ad7ca0dbc654da9267474644a41,23708908,Constitutive JAK/STAT3 signaling contributes t...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,None,c856627242a754d2d756b32843523e6d7a089148,13232625,Summary: The current work characterizes young ...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,None,d4c9b2fa2b760b5cf90ce8635a7dede5b4cd58a2,73484844,Ionotropic glutamate receptors (iGluRs) mediat...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,None,7f13abe2c82bf0c66ca423e905d8f5967c4517b1,229159752,Aim The current pandemic of coronavirus diseas...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729812,None,3d52d2996d04fc19579c229a7463043528cfd40c,238585800,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
729813,None,73a33964257197838dd650591033e53a65ae086d,20384358,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
729814,None,0a13b06645128dc58a80bc02829f1b1d1c04ab2f,2264851,"Previously, extracellular vesicle production i...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
729815,None,6f024884dee1207774c3e685b440ba66f34f9668,7091622,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
from sqlalchemy import MetaData

# Query to pull in corpus IDs

sql = '''
    SELECT * FROM attributes;
'''

# Context manager to open the connection to the database and execute the query. 
with engine.connect() as conn:
    query = conn.execute(text(sql))
    md = MetaData(query)
    
# Get all records and store in dataframe
# df = pd.DataFrame(query.fetchall())
# df.head()